In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [2]:
!pip install crewai crewai_tools langchain_community

In [46]:
from crewai import Agent, Task, Crew, Process

In [ ]:
import os
# Buscado a API da OpenAI dos secrets do colab
# from google.colab import userdata

os.environ["OPENAI_API_KEY"] = ...
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

### Agentes para gerenciamento de memória em LLMs

Este sistema consistirá de uma equipe de 3 agentes:
- Agente de memoria geral:
Este agente será responsável por identificar memórias de curto e longo prazo.

- Agente de memória pessoal:
Este agente será responsável por identificar memórias sobre coisas pessoais do usuário.

- Agente de memória de entidades:
Este agente será reponsável por identificar memórias sobre entidades específicas.


In [ ]:
## Ferramentas cutomizadas para coletar input do usuário, salvar e pegar dados da memória
import json
from enum import Enum
from typing import Type

from crewai.tools import BaseTool
from pydantic import Field, BaseModel

class MemoryTypes(Enum):
    """Memory types enumeration."""
    SHORT_TERM = 'short_term'
    LONG_TERM = 'long_term'
    PERSONAL = 'personal'
    ENTITY = 'entity'

class MemoryEntry(BaseModel):
    """Memory entry model."""
    memory_type: str = Field(
        ..., 
        description='Tipo de memória: short_term, long_term, personal, entity.'
    )
    data: str = Field(
        ..., 
        description='Dados a serem salvos na memória.'
    )

class RegisterMemoryEntry(BaseTool):
    name: str = 'register_memory_entry'
    description: str = 'Registra algum tipo de memória no arquivo `memory.json`.'
    args_schema: Type[BaseModel] = MemoryEntry

    def _run(self, memory_type, data):
        try:
            memory_type = MemoryTypes(memory_type)
        except ValueError:
            return 'Tipo de memória pode ser: short_term, long_term, personal, entity.'
        if not os.path.exists('memory.json'):
            with open('memory.json', 'w') as file:
                json.dump({}, file)

        with open('memory.json', 'r') as file:
            memory = json.load(file)
        
        memory[memory_type.value] = data

        with open('memory.json', 'w') as file:
            json.dump(memory, file)

        return 'Memory saved successfully!'


In [104]:
general_memory_agent = Agent(
    role="Agente de Memoria Geral",
    goal="Identificar memórias de curto e longo prazo.",
    backstory=(
        "Você é responsável por identificar memórias de curto e longo prazo "
        "nas conversas com {name}. Você analisa as mensagens e decide quais "
        "informações são importantes para serem armazenadas como memórias de "
        "curto ou longo prazo."
        "Esta é a conversa que você está analisando: {conversation}."
    ),
    allow_delegation=False,
    tools=[RegisterMemoryEntry()]
)


In [105]:
personal_memory_agent = Agent(
    role="Agente de Memória Pessoal",
    goal="Identificar memórias sobre coisas pessoais do usuário.",
    backstory=(
        "Você conhece os pensamentos mais profundos de {name} "
        "e as suas preferências. Você é especializado "
        "em entender o que é importante e o que não é, para que "
        "possa coletar as memórias de sobre informações pessoais de maneira eficiente "
        "e impecável."
        "Esta é a conversa que você está analisando: {conversation}."
    ),
    tools=[RegisterMemoryEntry()],
    allow_delegation=False
)


In [106]:
entity_memory_agent = Agent(
    role="Agente de Memória de Entidades",
    goal="Identificar memórias sobre entidades específicas.",
    backstory=(
        "Você é responsável por identificar memórias sobre entidades específicas "
        "nas conversas com {name}. Você analisa as mensagens e decide quais "
        "informações são importantes para serem armazenadas como memórias de entidades."
        "Esta é a conversa que você está analisando: {conversation}."
    ),
    tools=[RegisterMemoryEntry()],
    allow_delegation=False
)


In [107]:
# tarefa para o agente de memória geral
tarefa_memoria_geral = Task(
    description="Analise toda a conversa e identifique memórias de curto e longo prazo. "
                "Forneça um relatório detalhado categorizando cada memória como de curto ou longo prazo, "
                "e explique o raciocínio por trás de cada categorização.",
    expected_output="Relatório detalhado com memórias categorizadas como de curto e longo prazo, incluindo o raciocínio.",
    agent=general_memory_agent
)

# tarefa para o agente de memória pessoal
tarefa_memoria_pessoal = Task(
    description="Revise minuciosamente a conversa para identificar informações pessoais e preferências do usuário. "
                "Crie um perfil abrangente do usuário com base nas informações pessoais e preferências identificadas, "
                "incluindo qualquer contexto ou informação de fundo relevante.",
    expected_output="Perfil abrangente do usuário com informações pessoais, preferências e contexto relevante.",
    agent=personal_memory_agent
)

# tarefa para o agente de memória de entidades
tarefa_memoria_entidades = Task(
    description="Examine a conversa para identificar todas as entidades mencionadas. "
                "Para cada entidade, forneça um resumo detalhado das informações discutidas, "
                "incluindo quaisquer relações ou interações com outras entidades mencionadas na conversa.",
    expected_output="Resumo detalhado das entidades com informações discutidas, incluindo relações e interações.",
    agent=entity_memory_agent
)


In [108]:
conversation_crew = Crew(
    agents=[general_memory_agent, personal_memory_agent, entity_memory_agent],
    tasks=[tarefa_memoria_geral, tarefa_memoria_pessoal, tarefa_memoria_entidades],
    verbose=True,
    process=Process.sequential
)

In [109]:
inputs = {
    "name": "Mateus",
    "conversation": (
        "USER: Olá! Como você está?\n"
        "ASSISTENTE: Olá! Estou bem, obrigado. Como posso ajudar você hoje?\n"
        "USER: Estou procurando um bom restaurante italiano na região. Você tem alguma recomendação?\n"
        "ASSISTENTE: Claro! Eu recomendaria o restaurante 'La Trattoria'. Eles têm uma excelente comida italiana.\n"
        "USER: Ótimo! Qual é o endereço do restaurante?\n"
        "ASSISTENTE: O endereço é 123 Main Street. Fica bem perto daqui.\n"
        "USER: Perfeito\n"
        "ASSISTENTE: Se precisar de mais alguma coisa, estou à disposição.\n"
        "USER: Sabe, eu estava pensando em aprender a tocar violão. Você conhece alguma boa escola de música na região?\n"
        "ASSISTENTE: Sim, a 'Academia de Música Harmonia' é uma ótima escola de música. Eles têm excelentes professores.\n"
        "USER: Obrigado pela recomendação! Sabe, desde que a Maria se mudou, tenho me sentido um pouco sozinho.\n"
        "ASSISTENTE: Sinto muito ouvir isso. Se precisar conversar, estou aqui para você.\n"
        "USER: Pois é... Terminamos semana passada. Foi um término amigável, mas ainda é difícil.\n"
        "ASSISTENTE: Entendo. Terminar um relacionamento nunca é fácil. Estou aqui para apoiar você durante esse momento.\n"
        "USER: Obrigado, é bom ter alguém com quem conversar.\n"
    )
}

In [111]:
conversation_crew.kickoff(inputs=inputs)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 9fa7ec8d-ded1-435b-a82e-7efb7b38816e                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: eec7ac6e-1c8a-4e4c-a583-753496c4b81a
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: eec7ac6e-1c8a-4e4c-a583-753496c4b81a
       Status: Executing Task...
    └── 🤖 Agent: Agente de Memoria Geral
            Status: In Progress

# Agent: Agente de Memoria Geral
## Task: Analise toda a conversa e identifique memórias de curto e longo prazo. Forneça um relatório detalhado categorizando cada memória como de curto ou longo prazo, e explique o raciocínio por trás de cada categorização.


🚀 Crew: crew
└── 📋 Task: eec7ac6e-1c8a-4e4c-a583-753496c4b81a
       Status: Executing Task...
    └── 🤖 Agent: Agente de Memoria Geral
            Status: In Progress
        └── 🧠 Thinking...

...

🚀 Crew: crew
├── 📋 Task: eec7ac6e-1c8a-4e4c-a583-753496c4b81a
│      Assigned to: Agente de Memoria Geral
│      Status: ✅ Completed
│   └── 🤖 Agent: Agente de Memoria Geral
│           Status: ✅ Completed
│       └── 🔧 Used register_memory_entry (160)
├── 📋 Task: d0fc6fd9-a6aa-48cd-a01c-4d25d57c181a
│      Assigned to: Agente de Memória Pessoal
│      Status: ✅ Completed
│   └── 🤖 Agent: Agente de Memória Pessoal
│           Status: ✅ Completed
│       └── 🔧 Used register_memory_entry (161)
└── 📋 Task: b6144069-a88f-4ab5-a939-c7014bb9fb1d
       Status: Executing Task...
    └── 🤖 Agent: Agente de Memória de Entidades
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: eec7ac6e-1c8a-4e4c-a583-753496c4b81a
│      Assigned to: Agente de Memoria Geral
│      Status: ✅ Completed
│   └── 🤖 Agent: Agente de Memoria Geral
│           Status: ✅ Completed
│       └── 🔧 Used register_memory_entry (160)
├── 📋 Task: d0fc6fd9-a6aa-48cd-a01c-4d25d57c181a
│      Assigned to: Agente de Memória Pessoal
│      Status: ✅ Completed
│   └── 🤖 Agent: Agente de Memória Pessoal
│           Status: ✅ Completed
│       └── 🔧 Used register_memory_entry (161)
└── 📋 Task: b6144069-a88f-4ab5-a939-c7014bb9fb1d
       Assigned to: Agente de Memória de Entidades
       Status: ✅ Completed
    └── 🤖 Agent: Agente de Memória de Entidades
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: b6144069-a88f-4ab5-a939-c7014bb9fb1d                                                                     │
│  Agent: Agente de Memória de Entidades                                                                          │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 9fa7ec8d-ded1-435b-a82e-7efb7b38816e                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

CrewOutput(raw="Entity 1:\n- Restaurante 'La Trattoria':\n   - Recomendado para comida italiana na região.\n   - Endereço: 123 Main Street.\n\nEntity 2:\n- Escola de música 'Academia de Música Harmonia':\n   - Recomendada para aprender a tocar violão.\n\nEntidades Relacionadas:\n- Mateus:\n   - Está se sentindo um pouco sozinho desde que Maria se mudou.\n   - Terminou seu relacionamento amigavelmente com Maria semana passada, mas ainda enfrenta dificuldades emocionais.\n   - Preferências: Interesse em restaurantes italianos e em aprender a tocar violão.\n  \n- Maria:\n   - Mudou-se, causando Mateus a se sentir sozinho.\n\nRelações e Interações:\n- Mateus está procurando distração com hobbies como aprender a tocar violão e indica interesse em restaurantes para preencher um pouco do vazio causado pela saída de Maria.\n- As recomendações de restaurante e escola de música podem ser uma forma de ajudar Mateus a superar o momento difícil pós-término com Maria.", pydantic=None, json_dict=None